In [ ]:
import struct
import random
import numpy as np
# import open3d as o3d
# import matplotlib as mpl
# import matplotlib.cm as cm

from mayavi import mlab

class TDA_IO(object):
    
    @staticmethod
    def read_dat(path):
        '''
        path: path to the .dat binary file
        Note: the dat file has file identifier (pengxiang's version)
        '''
        fileIn = open(path, "rb")
        fileID = struct.unpack('I', fileIn.read(4))[0]
        dims   = struct.unpack('I', fileIn.read(4))[0]
        shape  = struct.unpack(str(dims)+'I', fileIn.read(4 * dims))
        dat    = struct.unpack(str(np.prod(shape))+'d', fileIn.read(8*np.prod(shape)))
        dat    = np.reshape(dat, shape)
        fileIn.close()
        return dat
    
    @staticmethod
    def cvt_dat2cloud(dat):
        shape = dat.shape
        min_val = np.amin(dat)
        valid_pts = np.prod(shape) - np.sum(dat==min_val)
        pt_cloud = np.zeros((valid_pts, 3),dtype=int)
        
#         norm = mpl.colors.Normalize(vmin=np.amin(dat), vmax=np.amax(dat))
#         cmap = cm.hot
#         m = cm.ScalarMappable(norm=norm, cmap=cmap)
        
        cnt = 0
        for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    if dat[i][j][k] > min_val:
                        #color = m.to_rgba(dat[i][j][k])
                        val_ = [i, j, k]#, color[0], color[1], color[2], random.uniform(-1, 1), random.uniform(-1, 1), random.uniform(-1, 1)]
                        pt_cloud[cnt,:] = val_
                        cnt = cnt + 1
        return pt_cloud

path2 = 'E:/Data2/BreastMass_processed/1/inn/ISPY_1001_inn_1.dat'
dat = TDA_IO.read_dat(path2)

hollow_num = np.sum(dat == np.amin(dat))
print(hollow_num/np.prod(dat.shape))


# # point_cloud = TDA_IO.cvt_dat2cloud(dat)
# # # mlab.plot3d(point_cloud[:,0],point_cloud[:,1],point_cloud[:,2])


# # x = point_cloud[:,0]
# # y = point_cloud[:,1]
# # z = point_cloud[:,2]
# # d = np.sqrt(x ** 2 + y ** 2)
# # fig = mlab.figure(bgcolor=(0, 0, 0), size=(640, 360))
# # mlab.plot3d(x, y, z
# #              )
# # mlab.show()


# mlab.figure(1, bgcolor=(1, 1, 1), size=(350, 350))
# source = mlab.pipeline.scalar_field(dat)
# vol = mlab.pipeline.volume(source)
# #mlab.view(132, 54, 45, [21, 20, 21.5])

# mlab.show()

        
# path = 'D:/WorkBench/Lazarus01/sample.xyz'
# point_cloud = np.loadtxt(path, skiprows=1)
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
# pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6])
# pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

# distances = pcd.compute_nearest_neighbor_distance()
# avg_dist = np.mean(distances)
# radius = 3 * avg_dist
# bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

#poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=5, width=0, scale=1.1, linear_fit=False)[0]

# o3d.visualization.draw_geometries([pcd])

In [ ]:
import numpy as np
import random
import open3d as o3d

path = 'D:/WorkBench/Lazarus01/sample_w_normals.xyz'
point_cloud= np.loadtxt(path,skiprows=1)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
# pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.colors = o3d.utility.Vector3dVector(np.zeros((point_cloud.shape[0], 3)))
# pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

normals = np.zeros((point_cloud.shape[0], 3))
for i in range(point_cloud.shape[0]):
    normals[i, 0] = random.uniform(-1, 1)
    normals[i, 1] = random.uniform(-1, 1)
    normals[i, 2] = random.uniform(-1, 1)
pcd.normals = o3d.utility.Vector3dVector(normals)

poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]
bbox = pcd.get_axis_aligned_bounding_box()
poisson_mesh = poisson_mesh.crop(bbox)

o3d.visualization.draw_geometries([poisson_mesh])


In [ ]:
# Create the data.
from numpy import pi, sin, cos, mgrid
dphi, dtheta = pi/250.0, pi/250.0
[phi,theta] = mgrid[0:pi+dphi*1.5:dphi,0:2*pi+dtheta*1.5:dtheta]
m0 = 4; m1 = 3; m2 = 2; m3 = 3; m4 = 6; m5 = 2; m6 = 6; m7 = 4;
r = sin(m0*phi)**m1 + cos(m2*phi)**m3 + sin(m4*theta)**m5 + cos(m6*theta)**m7
x = r*sin(phi)*cos(theta)
y = r*cos(phi)
z = r*sin(phi)*sin(theta)

# View it.
from mayavi import mlab
s = mlab.mesh(x, y, z)
mlab.show()

In [ ]:
import numpy as np
from mayavi.mlab import quiver3d, draw

# Primitives
N = 200 # Number of points
ones = np.ones(N)
scalars = np.arange(N) # Key point: set an integer for each point

# Define color table (including alpha), which must be uint8 and [0,255]
colors = (np.random.random((N, 4))*255).astype(np.uint8)
colors[:,-1] = 255 # No transparency

# Define coordinates and points
x, y, z = colors[:,0], colors[:,1], colors[:,2] # Assign x, y, z values to match color
pts = quiver3d(x, y, z, scalars=scalars, mode='sphere') # Create points
pts.glyph.color_mode = 'color_by_scalar' # Color by scalar

# Set look-up table and redraw
pts.module_manager.scalar_lut_manager.lut.table = colors
draw()

In [ ]:
import numpy as np
from mayavi import mlab
mlab.figure(1, bgcolor=(0, 0, 0), size=(350, 350))
mlab.clf()

# The position of the atoms
atoms_x = np.array([2.9, 2.9, 3.8]) * 40 / 5.5
atoms_y = np.array([3.0, 3.0, 3.0]) * 40 / 5.5
atoms_z = np.array([3.8, 2.9, 2.7]) * 40 / 5.5

O = mlab.points3d(atoms_x[1:-1], atoms_y[1:-1], atoms_z[1:-1],
                  scale_factor=3,
                  resolution=20,
                  color=(1, 0, 0),
                  scale_mode='none')

H1 = mlab.points3d(atoms_x[:1], atoms_y[:1], atoms_z[:1],
                   scale_factor=2,
                   resolution=20,
                   color=(1, 1, 1),
                   scale_mode='none')

H2 = mlab.points3d(atoms_x[-1:], atoms_y[-1:], atoms_z[-1:],
                   scale_factor=2,
                   resolution=20,
                   color=(1, 1, 1),
                   scale_mode='none')

# The bounds between the atoms, we use the scalar information to give
# color
mlab.plot3d(atoms_x, atoms_y, atoms_z, [1, 2, 1],
            tube_radius=0.4, colormap='Reds')

# Display the electron localization function ##################################

# Load the data, we need to remove the first 8 lines and the '\n'
# str = ' '.join(file('h2o-elf.cube').readlines()[9:])
# data = np.fromstring(str, sep=' ')
# data.shape = (40, 40, 40)

# source = mlab.pipeline.scalar_field(data)
# min = data.min()
# max = data.max()
# vol = mlab.pipeline.volume(source, vmin=min + 0.65 * (max - min),
#                                    vmax=min + 0.9 * (max - min))

mlab.view(132, 54, 45, [21, 20, 21.5])

mlab.show()

In [5]:
import struct
import random
import numpy as np
# import open3d as o3d
# import matplotlib as mpl
# import matplotlib.cm as cm

from mayavi import mlab

class TDA_IO(object):
    
    @staticmethod
    def read_dat(path):
        '''
        path: path to the .dat binary file
        Note: the dat file has file identifier (pengxiang's version)
        '''
        fileIn = open(path, "rb")
        fileID = struct.unpack('I', fileIn.read(4))[0]
        dims   = struct.unpack('I', fileIn.read(4))[0]
        shape  = struct.unpack(str(dims)+'I', fileIn.read(4 * dims))
        dat    = struct.unpack(str(np.prod(shape))+'d', fileIn.read(8*np.prod(shape)))
        dat    = np.reshape(dat, shape)
        fileIn.close()
        return dat
    
    @staticmethod
    def cvt_dat2cloud(dat):
        shape = dat.shape
        min_val = np.amin(dat)
        valid_pts = np.prod(shape) - np.sum(dat==min_val)
        pt_cloud = np.zeros((valid_pts, 3),dtype=int)
        
#         norm = mpl.colors.Normalize(vmin=np.amin(dat), vmax=np.amax(dat))
#         cmap = cm.hot
#         m = cm.ScalarMappable(norm=norm, cmap=cmap)
        
        cnt = 0
        for i in range(shape[0]):
            for j in range(shape[1]):
                for k in range(shape[2]):
                    if dat[i][j][k] > min_val:
                        #color = m.to_rgba(dat[i][j][k])
                        val_ = [i, j, k]#, color[0], color[1], color[2], random.uniform(-1, 1), random.uniform(-1, 1), random.uniform(-1, 1)]
                        pt_cloud[cnt,:] = val_
                        cnt = cnt + 1
        return pt_cloud

# path2 = 'E:/Data2/BreastMass_processed/1/inn/ISPY_1001_inn_1.dat'
# data = TDA_IO.read_dat(path2)
# data = data + np.abs(np.amin(data)) + 1


import numpy as np
import os
data = np.array([np.fromfile(os.path.join('D:/WorkBench/Lazarus01/mri_data', 'MRbrain.%i' % i),
                                        dtype='>u2') for i in range(1, 110)])
data.shape = (109, 256, 256)
data = data.T

from mayavi import mlab

mlab.figure(bgcolor=(0, 0, 0), size=(400, 400))

src = mlab.pipeline.scalar_field(data)
# Our data is not equally spaced in all directions:
src.spacing = [1, 1, 1.5]
src.update_image_data = True


# Extract some inner structures: the ventricles and the inter-hemisphere
# fibers. We define a volume of interest (VOI) that restricts the
# iso-surfaces to the inner of the brain. We do this with the ExtractGrid
# filter.
blur = mlab.pipeline.user_defined(src, filter='ImageGaussianSmooth')
voi = mlab.pipeline.extract_grid(blur)
voi.trait_set(x_min=125, x_max=193, y_min=92, y_max=125, z_min=34, z_max=75)
mlab.pipeline.iso_surface(voi, contours=[1610, 2480], colormap='Spectral')

# Add two cut planes to show the raw MRI data. We use a threshold filter
# to remove cut the planes outside the brain.
thr = mlab.pipeline.threshold(src, low=1120)
cut_plane = mlab.pipeline.scalar_cut_plane(thr,
                                plane_orientation='y_axes',
                                colormap='black-white',
                                vmin=1400,
                                vmax=2600)
cut_plane.implicit_plane.origin = (136, 111.5, 82)
cut_plane.implicit_plane.widget.enabled = False

cut_plane2 = mlab.pipeline.scalar_cut_plane(thr,
                                plane_orientation='z_axes',
                                colormap='black-white',
                                vmin=1400,
                                vmax=2600)
cut_plane2.implicit_plane.origin = (136, 111.5, 82)
cut_plane2.implicit_plane.widget.enabled = False

# Extract two views of the outside surface. We need to define VOIs in
# order to leave out a cut in the head.
voi2 = mlab.pipeline.extract_grid(src)
voi2.trait_set(y_min=112)
outer = mlab.pipeline.iso_surface(voi2, contours=[1776, ], color=(0.8, 0.7, 0.6), opacity=1.0)

voi3 = mlab.pipeline.extract_grid(src)
voi3.trait_set(y_max=112, z_max=53)
outer3 = mlab.pipeline.iso_surface(voi3, contours=[1776, ],
                                         color=(0.8, 0.7, 0.6))

mlab.view(-125, 54, 326, (145.5, 138, 66.5))
mlab.roll(-175)

mlab.show()